In [1]:
import os            # linux shell
import re            #^[]?%
import pymatgen as mg  #materials project
import pymatgen.analysis.diffraction as anadi
import pymatgen.analysis.diffraction.xrd as xrd
import numpy as np
import glob          #random
import matplotlib.pyplot as plt
import torch         #machine learning
import torch.nn as nn
from torch.autograd import Variable

In [2]:
torch.set_default_dtype(torch.float64)

torch.set_printoptions(precision=16)

patt_xrd = xrd.XRDCalculator('CuKa')

train_path='/home/mii/trainCSSO/O3/train/'

test_path='/home/mii/trainCSSO/O3/test/'

global sample_num, rmat_num, atoms_num
sample_num=1
rmat_num=28  #row nums of the matrix for the input of CNN 
atoms_num=24

global move_num,extend_num
extend_num=100

LR_D=0.001  #learning rate
LR_G=0.001

In [3]:
move_num=-115

In [4]:
def random_xxpsk(file_path):
    folder=np.random.choice(glob.glob(file_path +"*"))
    return folder

In [5]:
def tomgStructure(folder):
    POSfile=folder+'/CONTCAR'
    R_mgS=mg.Structure.from_file(POSfile)
    return R_mgS
    #print(POSfile)
    '''
    with open(POSfile,'r+') as pp:
        transfer=pp.readlines()
        if transfer[5] =='   Ca   Sn   S    O \n':
            pass
        else:
            transfer.insert(5,'Ca   Sn   S    O\n')
            pp.seek(0)# move the zhizhen
            pp.truncate()#read by the byte
            pp.writelines(transfer)
        pp.close()
    '''   

In [6]:
def get_xrdmat4(mgStructure):
    global rmat_num
    xrd_data4 =patt_xrd.get_pattern(mgStructure)
    i_column = rmat_num
    xxx=[]
    yyy=[]
    mat4=[]
    xrd_i=len(xrd_data4)
    for i in range(xrd_i):
        if xrd_data4.y[i]>1 and xrd_data4.y[i] < 20:
            xxx.append(xrd_data4.x[i])
            yyy.append(xrd_data4.y[i])
    mat4.append(np.asarray(xxx))
    mat4.append(np.asarray(yyy))
    mat4=np.asarray(mat4)
    
    xrd_x=[]
    xrd_y=[]
    xrd_mat4=[]
    xrow=len(mat4[0])
    
    if xrow < i_column:
        for i in mat4[0]:
            xrd_x.append(i)
        for j in mat4[1]:
            xrd_y.append(j)
        for i in range(0,i_column-xrow):
            xrd_x.append(0)
            xrd_y.append(0)
        xrd_x=np.asarray(xrd_x)
        xrd_y=np.asarray(xrd_y)
    if xrow > i_column:
        xrd_x=mat4[0][:i_column]
        xrd_y=mat4[1][:i_column]
    if xrow == i_column:
        xrd_x= mat4[0]
        xrd_y= mat4[1]
        
    xrd_x=np.sin(np.dot(1/180*np.pi,xrd_x))
    xrd_y=np.dot(1/100,xrd_y)
    xrd_mat4.append(xrd_x)
    xrd_mat4.append(xrd_y)
    xrd_mat4=np.array(xrd_mat4)
    return xrd_mat4

In [7]:
def get_energy(folder):
    energy_string=os.popen('grep TOTEN '+folder+'/OUTCAR | tail -1').read().split(' ')[-2]
    energy=np.float64(float(energy_string))
    return energy

def linear_transform(energy):
    global extend_num, move_num
    energy_transform=(energy-move_num)*extend_num
    return energy_transform
def inverse_transform(energy_transform):
    global extend_num, move_num
    energy=energy_transform/extend_num+move_num
    return energy
def get_energy_per_atom(energy):
    energy_per_atom=energy/atoms_num
    return energy_per_atom

In [ ]:
get_energy(train_path+'000')

In [8]:
def GANs_Gmat(Random_Structure):
    global rmat_num
    RS_xrdmat = get_xrdmat4(Random_Structure)
    multimat3_RS =  np.zeros((rmat_num,rmat_num),dtype='float32')
    multimat3_RS = np.asarray(np.sqrt(np.dot(RS_xrdmat.T, RS_xrdmat)))
    return multimat3_RS

In [9]:
class GcssNet(nn.Module):
    def __init__(self, input_size=(sample_num,28,28)):
        super(GcssNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(#(3,28,28)
                in_channels=sample_num,
                out_channels=32,
                kernel_size=5,
                stride=1,
                padding=2,
            ),#->(32,28,28)
            nn.ReLU(),#->(32,28,28)
            nn.MaxPool2d(kernel_size=2),
        )#->(#->(32,14,14))
        self.conv2=nn.Sequential(#->(32,14,14))
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=5,
                stride=1,
                padding=2,
            ),#->(64,14,14)
            nn.ReLU(),#->(64,14,14)
            nn.MaxPool2d(kernel_size=2),#->(64,7,7)
        )
        self.out=nn.Sequential(
            nn.Linear(64*7*7,128),
            nn.ReLU(),
            nn.Linear(128,sample_num),            
        )
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x) #batch(64,7,7)
        x=x.view(x.size(0),-1) #(batch, 64*7*7)
        output=torch.unsqueeze(self.out(x),dim=0)
        return output

class DNet(nn.Module):
    def __init__(self):
        super(DNet, self).__init__()
        self.Dlstm=nn.LSTM(
            input_size=sample_num,
            hidden_size=32,
            num_layers=1,
            batch_first=True,
        )
        self.out=nn.Sequential(
            nn.Linear(32,10),
            nn.ReLU(),
            nn.Linear(10,1),
            nn.Sigmoid(),
        )
        #nn.Linear(32,1)
        #nn.Relu
        #nn.Linear
        #nn.Sigmoid
        
    def forward(self,x):
        D_out,(h_n,h_c)=self.Dlstm(x,None)
        out = self.out(D_out[:,-1,:]) #(batch,time step,input)   
        return out

In [10]:
G1=GcssNet()
D1=DNet()

mat_Gl=[]    
mat_Dl=[]
pre_dd=[]
pre_gg=[]

In [140]:
opt_D1=torch.optim.Adam(D1.parameters(),lr=0.00001)
opt_G1=torch.optim.Adam(G1.parameters(),lr=0.000002)

In [183]:
file_path=train_path
tfset=[]
for step in range(1,2):       
 
    sample_path=[]
    for i in range(1,sample_num + 1):
        path_ = random_xxpsk(file_path)
        sample_path.append(path_)
        tfset.append(path_)
    E_Gibbs=[]
    for path1_ in sample_path:
        try:
            total_energy=get_energy(path1_)
            tt_energy=linear_transform(total_energy)
            #print(samp_Gibbs)
        except:
            print(path1_)
            
        E_Gibbs.append(tt_energy)
        
    E_Gibbs=np.asarray(E_Gibbs,dtype=np.float64)       
    E_Gibbs=Variable(torch.from_numpy(E_Gibbs[np.newaxis,np.newaxis,:]),requires_grad=True)
    #print(tfactor.shape)
    
    g_in=[]
    for path2_ in sample_path:
        path2_=str(path2_)                
        
        try:
            tomgS=tomgStructure(path2_)
            #print(tomgS)
            gin=GANs_Gmat(tomgS)
            #print(gin)
        except:
            pass
        g_in.append(gin)
       
    g_in=np.asarray(g_in)
    g_in=g_in[np.newaxis,:,:,:] 
    g_in=np.asarray(g_in,dtype=np.float64) 
    g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
    Gout=G1(g_in)
    #print(Gout.shape)
    
    prob_Tfactor_mat0=D1(E_Gibbs)
    prob_G1_mat1=D1(Gout)
    
    D1_loss=-torch.mean(torch.log(prob_Tfactor_mat0)+torch.log(1.-prob_G1_mat1))
    G1_loss=torch.mean(torch.log(1.-prob_G1_mat1))
    
    
    if step%1==0:
        opt_D1.zero_grad()
        D1_loss.backward(retain_graph=True)
        opt_D1.step()
        
        opt_G1.zero_grad()
        G1_loss.backward()
        opt_G1.step()
    else:
        opt_D1.zero_grad()
        D1_loss.backward()
        opt_D1.step()
    


    if step%1==0:
        print(step)
        gg=G1_loss.data.numpy().mean()
        dd=D1_loss.data.numpy().mean()
        print(dd)
        print(gg)
        print(prob_Tfactor_mat0.data.numpy().mean())
        print(prob_G1_mat1.data.numpy().mean())
        mat_Gl.append(gg)
        mat_Dl.append(dd)
        pre_dd.append(prob_Tfactor_mat0.data.numpy().mean())
        pre_gg.append(prob_G1_mat1.data.numpy().mean())
    
    
    if step%1==0:
        print('----------------------------------test-------------------------')
        file_path_test=test_path
        E_Gibbs=[]
        E_Gmodel=[]
        for step_test in range(10):
            sample_path=[]
            for i in range(1,sample_num + 1):
                path_ = random_xxpsk(file_path_test)
                sample_path.append(path_)
            
            for path1_ in sample_path:
                try:
                    total_energy=get_energy(path1_)
                    tt_energy=linear_transform(total_energy)
            #print(samp_Gibbs)
                except:
                    print(path1_)
                    
            tt_energy=float(tt_energy)
            print('DFT',tt_energy)
            E_Gibbs.append(tt_energy)
        
                   

        #print(tfactor.shape)
    
            g_in=[]
            for path2_ in sample_path:
                path2_=str(path2_)                
        
                try:
                    tomgS=tomgStructure(path2_)
            #print(tomgS)
                    gin=GANs_Gmat(tomgS)
            #print(gin)
                except:
                    pass
                g_in.append(gin)
       
            g_in=np.asarray(g_in)
            g_in=g_in[np.newaxis,:,:,:] 
            g_in=np.asarray(g_in,dtype=np.float64) 
            g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
            Gout=G1(g_in)
        #print(Gout.shape)
    
            G_data=Gout.data.numpy().mean()
            print('G_predict',G_data)
            E_Gmodel.append(G_data)

        
            
            
            print('error',abs(tt_energy-G_data))
            
        print('------------------end-test----------------------------')
        print(abs(np.asarray(E_Gibbs)-np.asarray(E_Gmodel)).mean())
    else:
        pass
    
    if step%1==0:
        print('----------------------------------test-t-------------------------')
        file_path_test=train_path
        E_Gibbs=[]
        E_Gmodel=[]
        for step_test in range(10):
            sample_path=[]
            for i in range(1,sample_num + 1):
                path_ = random_xxpsk(file_path_test)
                sample_path.append(path_)
            
            for path1_ in sample_path:
                try:
                    total_energy=get_energy(path1_)
                    tt_energy=linear_transform(total_energy)
            #print(samp_Gibbs)
                except:
                    print(path1_)
                    
            tt_energy=float(tt_energy)
            print('DFT',tt_energy)
            E_Gibbs.append(tt_energy)
        
                   

        #print(tfactor.shape)
    
            g_in=[]
            for path2_ in sample_path:
                path2_=str(path2_)                
        
                try:
                    tomgS=tomgStructure(path2_)
            #print(tomgS)
                    gin=GANs_Gmat(tomgS)
            #print(gin)
                except:
                    pass
                g_in.append(gin)
       
            g_in=np.asarray(g_in)
            g_in=g_in[np.newaxis,:,:,:] 
            g_in=np.asarray(g_in,dtype=np.float64) 
            g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
            Gout=G1(g_in)
        #print(Gout.shape)
    
            G_data=Gout.data.numpy().mean()
            print('G_predict',G_data)
            E_Gmodel.append(G_data)

        
            
            
            print('error',abs(tt_energy-G_data))
            
        print('------------------end-test----------------------------')
        print(abs(np.asarray(E_Gibbs)-np.asarray(E_Gmodel)).mean())
    else:
        pass

1
1.3878819721629445
-0.6923537986643404
0.4988109196883374
0.49960315164686475
----------------------------------test-------------------------
DFT 463.1369460000002
G_predict 478.05425460270726
error 14.917308602707067
DFT 448.07008300000035
G_predict 473.1209499812499
error 25.050866981249555
DFT 432.33006899999964
G_predict 463.73092725797335
error 31.40085825797371
DFT 426.5377520000001
G_predict 485.26814527213355
error 58.730393272133426
DFT 459.1093909999998
G_predict 478.0454552192213
error 18.93606421922152
DFT 462.0248239999995
G_predict 472.82810767402975
error 10.80328367403024
DFT 460.39525999999995
G_predict 470.8469161761408
error 10.45165617614083
DFT 473.4660390000002
G_predict 472.87567499785735
error 0.5903640021428487
DFT 447.16728899999936
G_predict 483.2768225340271
error 36.10953353402772
DFT 459.10978900000003
G_predict 478.0454552192213
error 18.935666219221275
------------------end-test----------------------------
22.59259949388482
----------------------------

In [182]:
torch.save(G1.state_dict(),"/home/mii/trainCSSO/G1_o3_last.pkl") 
torch.save(D1.state_dict(),"/home/mii/trainCSSO/D1_o3_last.pkl")

In [170]:
E_Gibbs_test=[]
E_Gmodel_test=[]
abserrset=[]
err0set=[]
testfile=[]
for m1,n1,fname in os.walk(test_path):
    for ieach in n1:
        ieach=test_path+ieach
        testfile.append(ieach)
        
for path_ in testfile:
    try:
        GGG=get_energy(path_)
        GGG=get_energy_per_atom(GGG)
        E_Gibbs_test.append(GGG)
        g_in=[]
        tomgS=tomgStructure(path_)
        gin=GANs_Gmat(tomgS)
        g_in.append(gin)
        g_in=np.asarray(g_in)
        g_in=g_in[np.newaxis,:,:,:]
        g_in=np.asarray(g_in,dtype=np.float64)
        g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
        Gout=G1(g_in)
        G_data=Gout.data.numpy().mean()
        G_data=inverse_transform(G_data)
        G_data=get_energy_per_atom(G_data)
        E_Gmodel_test.append(G_data)
        #print(G_data)
        #print(GGG)
        abserr=abs(G_data-GGG)
        abserrset.append(abserr)
        err0=abs(abserr/GGG)
        err0set.append(err0)
    except:
        print(path_)

In [172]:
print(np.asarray(err0set).mean())

0.0018375856157299014


In [171]:
print(np.asarray(abserrset).mean())

0.008463889284160322


In [168]:
E_Gibbs_t=[]
E_Gmodel_t=[]
abs_t_errset=[]
err_t_0set=[]
testfile=[]
for m1,n1,fname in os.walk(train_path):
    for ieach in n1:
        ieach=train_path+ieach
        testfile.append(ieach)
#'''        
for path_ in testfile:
    try:
        GGG=get_energy(path_)
        GGG=get_energy_per_atom(GGG)
        E_Gibbs_t.append(GGG)
        g_in=[]
        tomgS=tomgStructure(path_)
        gin=GANs_Gmat(tomgS)
        g_in.append(gin)
        g_in=np.asarray(g_in)
        g_in=g_in[np.newaxis,:,:,:]
        g_in=np.asarray(g_in,dtype=np.float64)
        g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
        Gout=G1(g_in)
        G_data=Gout.data.numpy().mean()
        G_data=inverse_transform(G_data)
        G_data=get_energy_per_atom(G_data)
        E_Gmodel_t.append(G_data)
        #print(G_data)
        #print(GGG)
        abserr=abs(G_data-GGG)
        abs_t_errset.append(abserr)
        err0=abs(abserr/GGG)
        err_t_0set.append(err0)
    except:
        print(path_)

In [169]:
print(np.asarray(abs_t_errset).mean())

0.007890270583684775


In [173]:
print(np.asarray(err_t_0set).mean())

0.0017169742863022916


In [174]:
print(mat_Dl)

[1.4151079158608337, 1.4066474024476907, 1.3901770031375227, 1.3723607939750302, 1.3450454340847484, 1.322774490049035, 1.3128697493867647, 1.3056656444839738, 1.2991276217852992, 1.2926214951257804, 1.2848655193520182, 1.276950908574244, 1.270740599720983, 1.263694738698352, 1.2567364558246568, 1.253666244999191, 1.2481262151232069, 1.2431660758408407, 1.2424357081989812, 1.237255744752862, 1.2320086637382768, 1.2344540385871516, 1.2289330327853039, 1.222658944592919, 1.2288489442156376, 1.22326550918805, 1.21798653932801, 1.226563837976483, 1.2215653216935065, 1.2122825407543651, 1.2238891360723287, 1.2206081896171193, 1.2146433261886918, 1.2242484590239477, 1.2216191761372412, 1.216752360820207, 1.2300469774231444, 1.227224036898095, 1.2202255982188168, 1.2347512625295758, 1.225321706023865, 1.2224417939299976, 1.2372111883948866, 1.2275943358061099, 1.2251354704852138, 1.237538028424062, 1.235329735355027, 1.2378930995974593, 1.2533906965832775, 1.2507366416641341, 1.23833845258327

In [175]:
print(pre_dd)

[0.46119653265280436, 0.46532636387018733, 0.47327457179569804, 0.48208062628454024, 0.4956462983034937, 0.5070211268787664, 0.5124033157252511, 0.5162666269752624, 0.5197816860869693, 0.5236107711119311, 0.5277827483027298, 0.5319751088563024, 0.5359960117162692, 0.5397011662632855, 0.5432995626175582, 0.5461100294342935, 0.5486703979093038, 0.5511073580160908, 0.5533263527820419, 0.5551734615049196, 0.557060467968907, 0.5584542862739372, 0.5604473755590852, 0.5618354382466839, 0.5632407621902412, 0.5645253500757115, 0.5660019338971913, 0.5671682330616781, 0.5688457306369057, 0.5701008806530649, 0.5710454975535874, 0.5716719753861773, 0.572421770722174, 0.573367359085623, 0.5738679975270854, 0.5737160153205955, 0.5752341139172301, 0.5760147611970542, 0.5739229346523069, 0.5756479884753944, 0.575430637165561, 0.5728978257751206, 0.5738563672489156, 0.5726957488924902, 0.5720053228466446, 0.5752291911360481, 0.5678038228700097, 0.557589747579722, 0.5580038188449464, 0.5580176668086705, 

In [176]:
print(mat_Gl)

[-0.6411769071464765, -0.641631140651269, -0.6420974342824708, -0.6427168895039364, -0.6431527191858168, -0.6435718841622156, -0.6442265113767056, -0.6445337164744427, -0.6447812317753427, -0.6456148212281755, -0.6457949778547523, -0.6457923299750719, -0.6471120409541088, -0.6469550502858283, -0.6466420253519246, -0.6487314405701384, -0.6478688292505886, -0.6473404291668208, -0.6506284063656068, -0.6487810739377992, -0.646927178879008, -0.6518715238671445, -0.6499131035205886, -0.6461126581182548, -0.654800843506327, -0.6514955199187987, -0.6488287553227088, -0.6594645260530004, -0.6574193164160618, -0.6503405905429421, -0.6636027440510185, -0.6614182679136091, -0.656764128271397, -0.6680198066625874, -0.6662632975626306, -0.6611316089018116, -0.6770688110120607, -0.6756020453617002, -0.6649654463549218, -0.6824923263074838, -0.672685121728783, -0.6653939012340893, -0.681835043145581, -0.6701936532740941, -0.6665284885155294, -0.6845513041021603, -0.6693504331647381, -0.653761293199130

In [177]:
print(pre_gg)

[0.4733277851431906, 0.47356696298381507, 0.4738123781340224, 0.474138226869088, 0.4743673631025719, 0.4745876437242253, 0.4749314803967724, 0.4750927593485293, 0.4752226658445578, 0.4756599324196725, 0.47575438724883967, 0.47575299910769636, 0.47644439730725663, 0.47636219751117626, 0.47619826016576294, 0.4772915569191158, 0.4768404681701307, 0.47656395758201253, 0.4782821770733079, 0.47731750005562273, 0.4763476027855698, 0.4789303306808375, 0.47790885732785143, 0.4759209032824094, 0.4804544768510213, 0.4787343695890468, 0.47734242168693447, 0.48287183097198233, 0.48181311317627545, 0.47813199681781327, 0.48500739829915696, 0.4838811795987144, 0.48147349202558126, 0.4872771363874422, 0.48637574259030486, 0.4837332082800453, 0.4918958389638667, 0.49115002239830624, 0.4857087019614054, 0.4946440903247311, 0.48966357900604374, 0.48592900537895617, 0.4943118192083909, 0.4883905066606044, 0.4865119330375238, 0.4956835364556918, 0.48795892531422014, 0.4799141023182372, 0.4882920484252377, 

In [178]:
print(err0set)

[0.0024123405181674163, 0.0007630794921184636, 0.0017113033248540438, 0.0015373581358431815, 0.000877824706557913, 0.0021269881175642554, 0.0013649381932353331, 0.005317557526678862, 0.00043858937217579243, 0.00112338600293133, 0.0009950803935414346, 0.0015499357281382664, 0.00011312652278316021, 0.0005470004886126853, 0.00036876358791537316, 0.0007292988693811499, 0.0019969531374500558, 0.004202979690882756, 0.003375866334599665, 0.000995072157500133, 0.002850084596609076, 0.002797048529628877, 0.0036653644870880756, 0.0027970507829082228, 0.0011007369402039255, 0.001956192656280734, 4.0325425301876376e-05, 0.004202910922599264, 0.0021701188087728356, 0.0012379471697997909, 0.002116691731553251, 0.0011730774197008253, 0.003962710274068949, 0.0009918666827546056, 6.457439640673268e-05, 0.0014529886961303347, 0.005317219152945576, 0.0008285738925966507, 0.0008778287909007675, 0.00046591920827967207, 0.003668031536311705, 0.001684889646260763, 0.00028126627266575755, 0.004925279381931344

In [179]:
print(abserrset)

[0.011105312778326493, 0.0035033925669933907, 0.007872518883267787, 0.0070710807866536385, 0.00403336889397643, 0.009788873322536418, 0.006276931631630234, 0.0245349135643691, 0.002019729743942378, 0.00515359005969529, 0.004576541869404593, 0.007133789370374011, 0.0005192747457298807, 0.0025151049062914765, 0.0016949500893579383, 0.0033532937927951423, 0.009195298694315568, 0.01937412405066663, 0.01558142903840043, 0.004576503952737454, 0.013143248162790222, 0.012894285617994328, 0.016912532375522815, 0.012894296034660968, 0.005064365482404298, 0.008966616518253367, 0.00018527069649287853, 0.019373805717333603, 0.009987571107651583, 0.005694861306663412, 0.009751465426582229, 0.005397072700979777, 0.018290113551111098, 0.004561749602711274, 0.0002972618576748687, 0.006663450333731724, 0.02453334398103557, 0.00380873915138924, 0.004033387643976205, 0.002137427796536251, 0.016924990613667923, 0.0077548487336080996, 0.001292388503584263, 0.022696817406997027, 0.006366238487040654, 0.00662

In [180]:
print(abs_t_errset)

[0.0031768332817714295, 0.007872774299934271, 0.014982997014868715, 0.01832505968201925, 0.010312333978307997, 0.0013048786311857796, 0.012747893531278365, 0.001303103631185465, 0.014694934925115533, 0.0007349782612866562, 0.015198222896854574, 1.9487068663615048e-05, 0.00206949101637921, 0.008145403805492002, 0.015926214601694966, 0.0009075565681566644, 0.0011436125097183947, 0.004335585057039459, 0.004153028669172443, 0.010771410948081872, 0.007549520003093946, 0.0007349557612865709, 2.9100955090122227e-05, 0.003352733792794993, 0.009338788714416069, 0.0028061426734300454, 0.011595400432492298, 0.006664235750397651, 0.009837664309725547, 0.0015849592009349323, 0.0012224643276983471, 0.003836017822171023, 5.917425462609316e-05, 0.013035409372999851, 0.00697581553061255, 0.008946602596846098, 0.0006984577082853605, 0.001107641702956741, 0.018463670162779167, 0.003044094871072822, 0.00011098754529470511, 0.00756728112002758, 0.007132476453707071, 2.9481371756290287e-05, 0.01469493325844

In [181]:
print(err_t_0set)

[0.0006928262622889567, 0.0017113587515059229, 0.003261626040699652, 0.0039961385630407475, 0.002239425911105901, 0.00028369871598695953, 0.002778502342289179, 0.0002833129156968761, 0.0031833664705977743, 0.00015986520092809797, 0.0033107667999662817, 4.238821822885785e-06, 0.0004499028215343145, 0.0017736739215616745, 0.0034799005486284324, 0.00019755434875364655, 0.00024896354615145815, 0.0009425795536792782, 0.0009042393941845543, 0.002344163072367736, 0.0016446500223530746, 0.00015986030616829856, 6.325231201148495e-06, 0.0007291771652923185, 0.0020284403772461213, 0.0006117345614035369, 0.0025265749695479395, 0.001453160207859059, 0.0021371506867773915, 0.0003448284299044357, 0.00026612133376565397, 0.0008330923995239428, 1.2861585398007234e-05, 0.002828534325756794, 0.0015193747064059298, 0.0019431554968877642, 0.00015220281380916545, 0.00024079688482021988, 0.004026506048166123, 0.0006611898543833724, 2.415532552663888e-05, 0.0016424434611958187, 0.0015496509169525786, 6.407916